In [ ]:
# $ conda env list
# # conda environments:
# #
# base                  *  /g/data1a/ha3/fxz547/miniconda3
# py37                     /g/data1a/ha3/fxz547/miniconda3/envs/py37


# (base) fxz547@vdi-n11 /g/data1a/ha3/fxz547/Githubz/passive-seismic/seismic/ml_classifier (fei/py3aws)
# $ conda activate py37

In [1]:
from keras import backend

Using TensorFlow backend.


In [3]:
import os
from random import shuffle,seed

def build_input_data(data_folder):
    """build a list of IDs and dictionary of labels
    """
    files=os.listdir(data_folder)
    IDs=[]
    for fname in files:
        if fname.endswith('.npy'):
            IDs.append(fname.rstrip('.npy'))

    labels={}
    Sctr=0
    Nctr=0
    for ID in IDs:
        if ID.endswith('_S'):
            labels[ID]=0
            Sctr+=1
        else:
            labels[ID]=1
            Nctr+=1

    seed(0)
    shuffle(IDs)
    trainLen=int(0.8*len(IDs))#use 80% of the data as training set
    valLen=int(0.1*len(IDs))
    partition={}
    partition['train'],partition['val'],partition['test']=IDs[0:trainLen],IDs[trainLen:trainLen+valLen],IDs[trainLen+valLen:]


    #create ID lists for S, N wave test cases

    inv_labels=[[],[]]
    for ID in partition['test']:
        inv_labels[labels[ID]].append(ID)

    print('dataset contains '+str(Sctr)+' S waves and '+str(Nctr)+' noise waveforms.')
    print('training set contains '+str(len(partition['train']))+' waveforms and test set contains '+str(len(partition['test']))+' waveforms.')
    
    return partition



In [5]:
#wave_data_folder='/g/data/ha3/fxz547/seiswave_data_4ml'
wave_data_folder='E:/Githubz/seiswave_data_4ml'

ds_parts=build_input_data(wave_data_folder)

print (ds_parts.keys())

dataset contains 14115 S waves and 13869 noise waveforms.
training set contains 22387 waveforms and test set contains 2799 waveforms.
dict_keys(['train', 'val', 'test'])


In [7]:
import data

SIDs=data.getIDs(0)
NIDs=data.getIDs(1)

print (SIDs[:10])
print(NIDs[:10])

dataset contains 14115 S waves and 13869 noise waveforms.
training set contains 22387 waveforms and test set contains 2799 waveforms.
['1776_S', '24049_S', '33438_S', '17198_S', '32385_S', '12547_S', '1938_S', '16143_S', '1045_S', '24837_S']
['18363_N', '23520_N', '27817_N', '1334_N', '14605_N', '25427_N', '27651_N', '1083_N', '2842_N', '3770_N']


In [9]:
import numpy as np
npyfile = os.path.join(wave_data_folder,"3996_S.npy")
my_waved= np.load(npyfile)

In [10]:
print (type(my_waved))

<class 'numpy.ndarray'>


In [11]:
import xarray as xar

xa = xar.DataArray(my_waved, dims=['wamplitude'])


In [12]:
xa.plot()

In [13]:
xa

<xarray.DataArray (wamplitude: 6000)>
array([ 0.      , -0.006568, -0.011815, ...,  0.011536,  0.006559,  0.      ])
Dimensions without coordinates: wamplitude

# Predict Phase Using a Trained Model

In [14]:
from model import shakenet
#from data import *
import data
import os
import matplotlib
matplotlib.use('Agg')
import obspy.core as oc

model=shakenet(pretrained_weights='shakenet-model.hdf5')


#dataDir='/g/data/ha3/rlt118/neural-datasets/categoriser-teleseismic/'
dataDir= data.datafolder  #'/g/data/ha3/fxz547/seiswave_data_4ml'
SIDs=data.getIDs(0)
NIDs=data.getIDs(1)

for ID in SIDs[:1]:
    x=np.resize(np.load(os.path.join(dataDir,ID+'.npy')),(1,6002,1))
    result=model.predict(x,batch_size=1)
    
    print (result)
    
    if np.argmax(result[0])==0:
        outfname='correctS/'+ID+'.png'
    else:
        outfname='badS-N/'+ID+'.png'
        
    trc=oc.stream.read(os.path.join(dataDir,ID+'.pkl'),  format="PICKLE")
        
    print("writing outfile", outfname)
    
    trc.plot(outfile=outfname)

for ID in NIDs[:1]:
    x=np.resize(np.load(os.path.join(dataDir,ID+'.npy')),(1,6002,1))
    result=model.predict(x,batch_size=1)
    print (result)
    
    trc=oc.stream.read(os.path.join(dataDir,ID+'.pkl'), format="PICKLE")
    if np.argmax(result[0])==0:
        outfname='badN-S/'+ID+'.png'
    else:
        outfname='correctN/'+ID+'.png'
        
    print("writing outfile", outfname) 
    trc.plot(outfile=outfname)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


E:\Githubz\passive-seismic\seismic\ml_classifier\model.py:34: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  model = Model(input = inputs, output = dense5)


[[  9.99993563e-01   6.41166525e-06]]


FileNotFoundError: [Errno 2] No such file or directory: 'E:/Githubz/seiswave_data_4ml\\1776_S.pkl'

In [16]:
!dir 'E:/Githubz/seiswave_data_4ml\\1776_S.pkl'

Invalid switch - "Githubz".
